### Imputing year build

In [17]:
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib as plt
import seaborn as sns
import sklearn
import math as m
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,confusion_matrix
from IPython.display import Image  
from sklearn.externals.six import StringIO  
from sklearn.tree import export_graphviz
import pydot 
import os
import statsmodels.api as sm

In [2]:
datapath= 'C:/Users/vincentlee/Desktop/datascienceprojects/Kaggle/ashrae/'

building_df = pd.read_csv(datapath + 'building_metadata.csv')
weather_train = pd.read_csv(datapath + "weather_train.csv")
weather_test = pd.read_csv(datapath + "weather_test.csv")
train = pd.read_csv(datapath + "train.csv")
test = pd.read_csv(datapath + 'test.csv')



In [3]:
print("train:", train.shape)
print("test:",test.shape)
print("building:",building_df.shape)
print("weathertrain:",weather_train.shape)
print("weathertest:",weather_test.shape)

train: (20216100, 4)
test: (41697600, 4)
building: (1449, 6)
weathertrain: (139773, 9)
weathertest: (277243, 9)


In [7]:
print("train: ", train.columns)
print("test: " , test.columns)
print("weather train: ", weather_train.columns)
print("weather test: ", weather_test.columns)
print("building meta: ", building_df.columns)


train:  Index(['building_id', 'meter', 'timestamp', 'meter_reading'], dtype='object')
test:  Index(['row_id', 'building_id', 'meter', 'timestamp'], dtype='object')
weather train:  Index(['site_id', 'timestamp', 'air_temperature', 'cloud_coverage',
       'dew_temperature', 'precip_depth_1_hr', 'sea_level_pressure',
       'wind_direction', 'wind_speed'],
      dtype='object')
weather test:  Index(['site_id', 'timestamp', 'air_temperature', 'cloud_coverage',
       'dew_temperature', 'precip_depth_1_hr', 'sea_level_pressure',
       'wind_direction', 'wind_speed'],
      dtype='object')
building meta:  Index(['site_id', 'building_id', 'primary_use', 'square_feet', 'year_built',
       'floor_count'],
      dtype='object')


In [4]:
# function to check how many null values
def nullvalues(dataframe):
    data_dict = {}
    for col in dataframe.columns:
        data_dict[col] = round(dataframe[col].isnull().sum()/dataframe.shape[0]*100,2)
    DataFrame = pd.DataFrame.from_dict(data_dict, orient='index', columns=['%missing_values'])
    return(DataFrame)

In [5]:
nullvalues(building_df)

,%missing_values
site_id,0.00
building_id,0.00
primary_use,0.00
square_feet,0.00
year_built,53.42
floor_count,75.50


In [6]:
nullvalues(weather_train)

,%missing_values
site_id,0.00
timestamp,0.00
air_temperature,0.04
cloud_coverage,49.49
dew_temperature,0.08
precip_depth_1_hr,35.98
sea_level_pressure,7.60
wind_direction,4.48
wind_speed,0.22


In [8]:
building_df.head()

,site_id,building_id,primary_use,square_feet,year_built,floor_count
0,7,799,Education,527431,1976.0,26.0
1,7,798,Education,409028,1970.0,21.0
2,7,802,Education,290625,1995.0,19.0
3,1,130,Lodging/residential,62893,1960.0,16.0
4,4,595,Education,270413,1971.0,14.0


In [9]:
building_df.describe()

,site_id,building_id,square_feet,year_built,floor_count
count,1449.000000,1449.000000,1449.000000,675.000000,355.000000
mean,6.952381,724.000000,92111.776398,1967.957037,3.740845
std,5.003432,418.434583,110769.950997,31.054030,3.333683
min,0.000000,0.000000,283.000000,1900.000000,1.000000
25%,3.000000,362.000000,23012.000000,1949.000000,1.000000
50%,5.000000,724.000000,57673.000000,1970.000000,3.000000
75%,13.000000,1086.000000,115676.000000,1995.000000,5.000000
max,15.000000,1448.000000,875000.000000,2017.000000,26.000000


In [13]:
site = building_df.groupby('site_id')

In [14]:
site.describe()

building_id                                                       \
              count    mean        std     min      25%     50%      75%   
site_id                                                                    
0             105.0    52.0  30.454885     0.0    26.00    52.0    78.00   
1              51.0   130.0  14.866069   105.0   117.50   130.0   142.50   
2             135.0   223.0  39.115214   156.0   189.50   223.0   256.50   
3             274.0   427.5  79.241193   291.0   359.25   427.5   495.75   
4              91.0   610.0  26.413380   565.0   587.50   610.0   632.50   
5              89.0   700.0  25.836021   656.0   678.00   700.0   722.00   
6              44.0   766.5  12.845233   745.0   755.75   766.5   777.25   
7              15.0   796.0   4.472136   789.0   792.50   796.0   799.50   
8              70.0   838.5  20.351085   804.0   821.25   838.5   855.75   
9             124.0   935.5  35.939764   874.0   904.75   935.5   966.25   
10             30.0  1012.5   8.803408   998.0  1005.25  1012.5  1019.75   
11              5.0  1030.0   1.581139  1028.0  1029.00  1030.0  1031.00   
12             36.0  1050.5  10.535654  1033.0  1041.75  1050.5  1059.25   
13            154.0  1145.5  44.600075  1069.0  1107.25  1145.5  1183.75   
14            102.0  1273.5  29.588849  1223.0  1248.25  1273.5  1298.75   
15            124.0  1386.5  35.939764  1325.0  1355.75  1386.5  1417.25   

                square_feet                 ... year_built          \
            max       count           mean  ...        75%     max   
site_id                                     ...                      
0         104.0       105.0   86394.638095  ...    2006.00  2016.0   
1         155.0        51.0   65002.392157  ...    1989.50  2007.0   
2         290.0       135.0   99542.214815  ...    1997.50  2014.0   
3         564.0       274.0   75290.383212  ...    2002.00  2017.0   
4         655.0        91.0   92876.241758  ...    1971.00  2016.0   
5         744.0        89.0   47612.415730  ...    1976.00  2013.0   
6         788.0        44.0   91857.704545  ...        NaN     NaN   
7         803.0        15.0  323634.533333  ...    1969.75  1995.0   
8         873.0        70.0   29712.200000  ...        NaN     NaN   
9         997.0       124.0  144874.209677  ...        NaN     NaN   
10       1027.0        30.0   71152.566667  ...        NaN     NaN   
11       1032.0         5.0  104563.400000  ...        NaN     NaN   
12       1068.0        36.0   79338.611111  ...        NaN     NaN   
13       1222.0       154.0  128342.090909  ...        NaN     NaN   
14       1324.0       102.0   95111.441176  ...        NaN     NaN   
15       1448.0       124.0   83921.387097  ...    1987.00  2014.0   

        floor_count                                                 
              count      mean       std  min  25%  50%   75%   max  
site_id                                                             
0               0.0       NaN       NaN  NaN  NaN  NaN   NaN   NaN  
1              51.0  7.000000  2.529822  2.0  5.5  7.0   8.0  16.0  
2               0.0       NaN       NaN  NaN  NaN  NaN   NaN   NaN  
3               0.0       NaN       NaN  NaN  NaN  NaN   NaN   NaN  
4              91.0  5.252747  2.702235  1.0  3.0  5.0   7.0  14.0  
5              89.0  2.280899  1.270189  1.0  1.0  2.0   3.0   8.0  
6               0.0       NaN       NaN  NaN  NaN  NaN   NaN   NaN  
7              15.0  8.733333  7.731814  1.0  3.0  6.0  12.0  26.0  
8              70.0  1.014286  0.119523  1.0  1.0  1.0   1.0   2.0  
9               0.0       NaN       NaN  NaN  NaN  NaN   NaN   NaN  
10             30.0  2.100000  1.184672  1.0  1.0  2.0   2.0   6.0  
11              0.0       NaN       NaN  NaN  NaN  NaN   NaN   NaN  
12              9.0  2.777778  1.301708  1.0  2.0  3.0   4.0   4.0  
13              0.0       NaN       NaN  NaN  NaN  NaN   NaN   NaN  
14              0.0       NaN       NaN  NaN  NaN  NaN   Na

In [16]:
primarysite = building_df.groupby('primary_use')
primarysite.describe()

site_id                                        \
                                count       mean       std  min   25%   50%   
primary_use                                                                   
Education                       549.0   6.384335  4.666214  0.0   3.0   5.0   
Entertainment/public assembly   184.0   6.668478  4.454247  0.0   3.0   5.0   
Food sales and service            5.0   9.000000  6.403124  2.0   2.0  13.0   
Healthcare                       23.0  10.086957  5.238954  2.0   3.0  13.0   
Lodging/residential             147.0   7.156463  5.832949  0.0   1.5   6.0   
Manufacturing/industrial         12.0  11.500000  4.011348  5.0  11.0  13.0   
Office                          279.0   8.648746  5.411012  0.0   3.0  12.0   
Other                            25.0   6.320000  4.308132  0.0   3.0   8.0   
Parking                          22.0   5.090909  5.698029  0.0   0.0   2.5   
Public services                 156.0   5.384615  3.797217  1.0   3.0   3.0   
Religious worship                 3.0   6.666667  7.234178  2.0   2.5   3.0   
Retail                           11.0   3.727273  5.405385  0.0   0.0   0.0   
Services                         10.0  11.000000  2.108185  9.0   9.0  11.0   
Technology/science                6.0   9.666667  4.676181  4.0   5.5  11.0   
Utility                           4.0   9.000000  6.976150  2.0   3.5   9.5   
Warehouse/storage                13.0   6.692308  4.785127  2.0   3.0   3.0   

                                           building_id               ...  \
                                 75%   max       count         mean  ...   
primary_use                                                          ...   
Education                       9.00  15.0       549.0   687.628415  ...   
Entertainment/public assembly   9.00  15.0       184.0   717.201087  ...   
Food sales and service         14.00  14.0         5.0   815.400000  ...   
Healthcare                     14.00  15.0        23.0   988.478261  ...   
Lodging/residential            14.00  15.0       147.0   730.673469  ...   
Manufacturing/industrial       13.50  15.0        12.0  1088.750000  ...   
Office                         13.00  15.0       279.0   829.756272  ...   
Other                           8.00  13.0        25.0   678.120000  ...   
Parking                        13.00  13.0        22.0   528.227273  ...   
Public services                 8.00  15.0       156.0   624.237179  ...   
Religious worship               9.00  15.0         3.0   692.333333  ...   
Retail                          7.50  12.0        11.0   363.545455  ...   
Services                       13.00  13.0        10.0  1043.800000  ...   
Technology/science             12.75  15.0         6.0   949.833333  ...   
Utility                        15.00  15.0         4.0   920.000000  ...   
Warehouse/storage              13.00  13.0        13.0   689.769231  ...   

                              year_built         floor_count            \
                                     75%     max       count      mean   
primary_use                                                              
Education                         1986.0  2017.0       172.0  4.732558   
Entertainment/public assembly     1991.0  2016.0        56.0  2.089286   
Food sales and service            2012.0  2012.0         0.0       NaN   
Healthcare                        2010.5  2014.0         1.0  1.000000   
Lodging/residential               2002.0  2013.0        18.0  6.333333   
Manufacturing/industrial          1974.5  1976.0         3.0  1.666667   
Office                            1994.0  2016.0        40.0  3.725000   
Other                             1987.0  1996.0        13.0  1.076923   
Parking                           2007.0  2014.0         3.0  2.666667   
Public services                   1981.0  2014.0        41.0  2.048780   
Religious worship                 1994.5  2010.0         0.0       NaN   
Retail                            2012.0  2016.